In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
import time
import itertools
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import os
import gym_Aircraft
from torch import Tensor
from typing import Type, Any, Callable, Union, List, Optional
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
env = gym.make("acav-v0")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/jupyter-kunwoopark/.local/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """transition 저장"""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [4]:
class FClayer(nn.Module):
    def __init__(self, innodes: int, nodes: int):
        super(FClayer, self).__init__()
        self.fc=nn.Linear(innodes,nodes)
        self.act=nn.LeakyReLU(0.2, inplace=True)
    def forward(self, x: Tensor) -> Tensor:
        out=self.fc(x)
        out=self.act(out)
        return out
    
class WaveNET(nn.Module):
    def __init__(self, block: Type[Union[FClayer]], planes: List[int], nodes: List[int], num_classes: int = 3
                ) -> None:
        super(WaveNET, self).__init__()
        self.innodes=5
        
        self.layer1=self._make_layer(block, planes[0], nodes[0])
        self.layer2=self._make_layer(block, planes[1], nodes[1])
        self.layer3=self._make_layer(block, planes[2], nodes[2])
        
        self.fin_fc=nn.Linear(self.innodes,num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
    
    def _make_layer(self, block: Type[Union[FClayer]], planes: int, nodes: int) -> nn.Sequential:

        layers = []
        layers.append(block(self.innodes, nodes))
        self.innodes = nodes
        for _ in range(1, planes):
            layers.append(block(self.innodes, nodes))

        return nn.Sequential(*layers)

        
    def _forward_impl(self, x: Tensor) -> Tensor:
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.fin_fc(x)
        
        return x
    
    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

In [5]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

In [6]:
policy_net = torch.load("./Custom_model_fin").to(device)
target_net = torch.load("./Custom_model_fin").to(device)
target_net.eval()

WaveNET(
  (layer1): Sequential(
    (0): FClayer(
      (fc): Linear(in_features=5, out_features=60, bias=True)
      (act): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (1): FClayer(
      (fc): Linear(in_features=60, out_features=60, bias=True)
      (act): LeakyReLU(negative_slope=0.2, inplace=True)
    )
  )
  (layer2): Sequential(
    (0): FClayer(
      (fc): Linear(in_features=60, out_features=20, bias=True)
      (act): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (1): FClayer(
      (fc): Linear(in_features=20, out_features=20, bias=True)
      (act): LeakyReLU(negative_slope=0.2, inplace=True)
    )
  )
  (layer3): Sequential(
    (0): FClayer(
      (fc): Linear(in_features=20, out_features=20, bias=True)
      (act): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (1): FClayer(
      (fc): Linear(in_features=20, out_features=20, bias=True)
      (act): LeakyReLU(negative_slope=0.2, inplace=True)
    )
  )
  (fin_fc): Linear(in_features=20, out_featu

In [7]:
n_actions = env.action_space.n

In [8]:
optimizer = optim.Adam(policy_net.parameters())
memory = ReplayMemory(50000)

In [9]:
steps_done = 0

In [10]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max (1)은 각 행의 가장 큰 열 값을 반환합니다.
            # 최대 결과의 두번째 열은 최대 요소의 주소값이므로,
            # 기대 보상이 더 큰 행동을 선택할 수 있습니다.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

In [11]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)

    batch = Transition(*zip(*transitions))

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_action_values = policy_net(state_batch).gather(1, action_batch)

    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # 기대 Q 값 계산
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Huber 손실 계산
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # 모델 최적화
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [ ]:
num_episodes = 50000
total_res=[]
reward_list=[]
for i_episode in range(num_episodes):
    total_reward=0
    
    # 환경과 상태 초기화
    res_list=np.zeros(11)
    state = env.reset()
    state=torch.from_numpy(state.astype(np.float32)).unsqueeze(0).to(device)
    for t in count():
        # 행동 선택과 수행
        
        action = select_action(state)
        next_state, reward, done, res = env.step(action.item())
        reward = torch.tensor([reward], dtype=torch.float32).to(device)
        
        next_state=torch.from_numpy(next_state.astype(np.float32)).unsqueeze(0).to(device)

        # 새로운 상태 관찰
        if not done:
            next_state = next_state
        else:
            next_state = None

        # 메모리에 변이 저장
        memory.push(state, action, next_state, reward)

        # 다음 상태로 이동
        state = next_state

        # 최적화 한단계 수행(목표 네트워크에서)
        optimize_model()
        
        # Data save
        
        cmd_list,r_list,elev_list,azim_list,Pm_list,Pt_list,h_list=res
        Pm_list=Pm_list.tolist()
        Pt_list=Pt_list.tolist()
        merged_data=itertools.chain([cmd_list],[r_list],[elev_list],[azim_list],Pm_list,Pt_list,[h_list])
        merged_data=np.array(list(merged_data))
        res_list=np.vstack([res_list,merged_data])
        
        total_reward+=reward
        
        if done:
            res_list=np.delete(res_list,0,0)
            
            total_res.append(res_list)
            reward_list.append(total_reward)
            
            now = time.localtime()
            print ("%04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec))
            print("episode : {} | final step : {} | total reward : {}".format(i_episode, t, total_reward.item()))
            break
            
        
    #목표 네트워크 업데이트, 모든 웨이트와 바이어스 복사
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.close()

2020/11/26 15:12:41
episode : 0 | final step : 98 | total reward : -1.0
2020/11/26 15:12:41
episode : 1 | final step : 98 | total reward : -1.0
2020/11/26 15:12:42
episode : 2 | final step : 98 | total reward : -1.0
2020/11/26 15:12:42
episode : 3 | final step : 98 | total reward : -1.0
2020/11/26 15:12:43
episode : 4 | final step : 98 | total reward : -1.0
2020/11/26 15:12:43
episode : 5 | final step : 98 | total reward : -1.0
2020/11/26 15:12:44
episode : 6 | final step : 98 | total reward : -1.0
2020/11/26 15:12:44
episode : 7 | final step : 98 | total reward : -1.0
2020/11/26 15:12:45
episode : 8 | final step : 98 | total reward : -1.0
2020/11/26 15:12:45
episode : 9 | final step : 98 | total reward : -1.0
2020/11/26 15:12:46
episode : 10 | final step : 98 | total reward : -1.0
2020/11/26 15:12:47
episode : 11 | final step : 98 | total reward : -1.0
2020/11/26 15:12:47
episode : 12 | final step : 98 | total reward : -1.0
2020/11/26 15:12:48
episode : 13 | final step : 98 | total re

2020/11/26 15:13:42
episode : 113 | final step : 98 | total reward : -1.0
2020/11/26 15:13:43
episode : 114 | final step : 98 | total reward : -1.0
2020/11/26 15:13:43
episode : 115 | final step : 98 | total reward : -1.0
2020/11/26 15:13:44
episode : 116 | final step : 98 | total reward : -1.0
2020/11/26 15:13:44
episode : 117 | final step : 98 | total reward : -1.0
2020/11/26 15:13:45
episode : 118 | final step : 98 | total reward : -1.0
2020/11/26 15:13:45
episode : 119 | final step : 98 | total reward : -1.0
2020/11/26 15:13:46
episode : 120 | final step : 98 | total reward : -1.0
2020/11/26 15:13:46
episode : 121 | final step : 98 | total reward : -1.0
2020/11/26 15:13:47
episode : 122 | final step : 98 | total reward : -1.0
2020/11/26 15:13:48
episode : 123 | final step : 98 | total reward : -1.0
2020/11/26 15:13:48
episode : 124 | final step : 98 | total reward : -1.0
2020/11/26 15:13:49
episode : 125 | final step : 98 | total reward : -1.0
2020/11/26 15:13:49
episode : 126 | fi

2020/11/26 15:14:43
episode : 224 | final step : 98 | total reward : -1.0
2020/11/26 15:14:43
episode : 225 | final step : 98 | total reward : -1.0
2020/11/26 15:14:44
episode : 226 | final step : 98 | total reward : -1.0
2020/11/26 15:14:45
episode : 227 | final step : 98 | total reward : -1.0
2020/11/26 15:14:45
episode : 228 | final step : 98 | total reward : -1.0
2020/11/26 15:14:46
episode : 229 | final step : 98 | total reward : -1.0
2020/11/26 15:14:46
episode : 230 | final step : 98 | total reward : -1.0
2020/11/26 15:14:47
episode : 231 | final step : 98 | total reward : -1.0
2020/11/26 15:14:47
episode : 232 | final step : 98 | total reward : -1.0
2020/11/26 15:14:48
episode : 233 | final step : 98 | total reward : -1.0
2020/11/26 15:14:48
episode : 234 | final step : 98 | total reward : -1.0
2020/11/26 15:14:49
episode : 235 | final step : 98 | total reward : -1.0
2020/11/26 15:14:50
episode : 236 | final step : 98 | total reward : -1.0
2020/11/26 15:14:50
episode : 237 | fi

2020/11/26 15:15:43
episode : 335 | final step : 98 | total reward : -1.0
2020/11/26 15:15:44
episode : 336 | final step : 98 | total reward : -1.0
2020/11/26 15:15:44
episode : 337 | final step : 98 | total reward : -1.0
2020/11/26 15:15:45
episode : 338 | final step : 98 | total reward : -1.0
2020/11/26 15:15:45
episode : 339 | final step : 98 | total reward : -1.0
2020/11/26 15:15:46
episode : 340 | final step : 98 | total reward : -1.0
2020/11/26 15:15:46
episode : 341 | final step : 98 | total reward : -1.0
2020/11/26 15:15:47
episode : 342 | final step : 98 | total reward : -1.0
2020/11/26 15:15:47
episode : 343 | final step : 98 | total reward : -1.0
2020/11/26 15:15:48
episode : 344 | final step : 98 | total reward : -1.0
2020/11/26 15:15:48
episode : 345 | final step : 98 | total reward : -1.0
2020/11/26 15:15:49
episode : 346 | final step : 98 | total reward : -1.0
2020/11/26 15:15:50
episode : 347 | final step : 98 | total reward : -1.0
2020/11/26 15:15:50
episode : 348 | fi

2020/11/26 15:16:43
episode : 446 | final step : 98 | total reward : -1.0
2020/11/26 15:16:44
episode : 447 | final step : 98 | total reward : -1.0
2020/11/26 15:16:44
episode : 448 | final step : 98 | total reward : -1.0
2020/11/26 15:16:45
episode : 449 | final step : 98 | total reward : -1.0
2020/11/26 15:16:46
episode : 450 | final step : 98 | total reward : -1.0
2020/11/26 15:16:46
episode : 451 | final step : 98 | total reward : -1.0
2020/11/26 15:16:47
episode : 452 | final step : 98 | total reward : -1.0
2020/11/26 15:16:47
episode : 453 | final step : 98 | total reward : -1.0
2020/11/26 15:16:48
episode : 454 | final step : 98 | total reward : -1.0
2020/11/26 15:16:48
episode : 455 | final step : 98 | total reward : -1.0
2020/11/26 15:16:49
episode : 456 | final step : 98 | total reward : -1.0
2020/11/26 15:16:49
episode : 457 | final step : 98 | total reward : -1.0
2020/11/26 15:16:50
episode : 458 | final step : 98 | total reward : -1.0
2020/11/26 15:16:50
episode : 459 | fi

...

2020/11/26 20:30:29
episode : 35066 | final step : 98 | total reward : -1.0
2020/11/26 20:30:30
episode : 35067 | final step : 98 | total reward : -1.0
2020/11/26 20:30:30
episode : 35068 | final step : 98 | total reward : -1.0
2020/11/26 20:30:31
episode : 35069 | final step : 98 | total reward : -1.0
2020/11/26 20:30:31
episode : 35070 | final step : 98 | total reward : -1.0
2020/11/26 20:30:32
episode : 35071 | final step : 98 | total reward : -1.0
2020/11/26 20:30:32
episode : 35072 | final step : 98 | total reward : -1.0
2020/11/26 20:30:33
episode : 35073 | final step : 98 | total reward : -1.0
2020/11/26 20:30:33
episode : 35074 | final step : 98 | total reward : -1.0
2020/11/26 20:30:34
episode : 35075 | final step : 98 | total reward : -1.0
2020/11/26 20:30:35
episode : 35076 | final step : 98 | total reward : -1.0
2020/11/26 20:30:35
episode : 35077 | final step : 98 | total reward : -1.0
2020/11/26 20:30:36
episode : 35078 | final step : 98 | total reward : -1.0
2020/11/26 2

2020/11/26 20:31:28
episode : 35174 | final step : 98 | total reward : -1.0
2020/11/26 20:31:29
episode : 35175 | final step : 98 | total reward : -1.0
2020/11/26 20:31:29
episode : 35176 | final step : 98 | total reward : -1.0
2020/11/26 20:31:30
episode : 35177 | final step : 98 | total reward : -1.0
2020/11/26 20:31:31
episode : 35178 | final step : 98 | total reward : -1.0
2020/11/26 20:31:31
episode : 35179 | final step : 98 | total reward : -1.0
2020/11/26 20:31:32
episode : 35180 | final step : 98 | total reward : -1.0
2020/11/26 20:31:32
episode : 35181 | final step : 98 | total reward : -1.0
2020/11/26 20:31:33
episode : 35182 | final step : 98 | total reward : -1.0
2020/11/26 20:31:33
episode : 35183 | final step : 98 | total reward : -1.0
2020/11/26 20:31:34
episode : 35184 | final step : 98 | total reward : -1.0
2020/11/26 20:31:34
episode : 35185 | final step : 98 | total reward : -1.0
2020/11/26 20:31:35
episode : 35186 | final step : 98 | total reward : -1.0
2020/11/26 2

2020/11/26 20:32:28
episode : 35282 | final step : 98 | total reward : -1.0
2020/11/26 20:32:28
episode : 35283 | final step : 98 | total reward : -1.0
2020/11/26 20:32:29
episode : 35284 | final step : 98 | total reward : -1.0
2020/11/26 20:32:29
episode : 35285 | final step : 98 | total reward : -1.0
2020/11/26 20:32:30
episode : 35286 | final step : 98 | total reward : -1.0
2020/11/26 20:32:30
episode : 35287 | final step : 98 | total reward : -1.0
2020/11/26 20:32:31
episode : 35288 | final step : 98 | total reward : -1.0
2020/11/26 20:32:31
episode : 35289 | final step : 98 | total reward : -1.0
2020/11/26 20:32:32
episode : 35290 | final step : 98 | total reward : -1.0
2020/11/26 20:32:33
episode : 35291 | final step : 98 | total reward : -1.0
2020/11/26 20:32:33
episode : 35292 | final step : 98 | total reward : -1.0
2020/11/26 20:32:34
episode : 35293 | final step : 98 | total reward : -1.0
2020/11/26 20:32:34
episode : 35294 | final step : 98 | total reward : -1.0
2020/11/26 2

2020/11/26 20:33:27
episode : 35390 | final step : 98 | total reward : -1.0
2020/11/26 20:33:28
episode : 35391 | final step : 98 | total reward : -1.0
2020/11/26 20:33:28
episode : 35392 | final step : 98 | total reward : -1.0
2020/11/26 20:33:29
episode : 35393 | final step : 98 | total reward : -1.0
2020/11/26 20:33:29
episode : 35394 | final step : 98 | total reward : -1.0
2020/11/26 20:33:30
episode : 35395 | final step : 98 | total reward : -1.0
2020/11/26 20:33:30
episode : 35396 | final step : 98 | total reward : -1.0
2020/11/26 20:33:31
episode : 35397 | final step : 98 | total reward : -1.0
2020/11/26 20:33:31
episode : 35398 | final step : 98 | total reward : -1.0
2020/11/26 20:33:32
episode : 35399 | final step : 98 | total reward : -1.0
2020/11/26 20:33:32
episode : 35400 | final step : 98 | total reward : -1.0
2020/11/26 20:33:33
episode : 35401 | final step : 98 | total reward : -1.0
2020/11/26 20:33:34
episode : 35402 | final step : 98 | total reward : -1.0
2020/11/26 2

2020/11/26 20:34:26
episode : 35498 | final step : 98 | total reward : -1.0
2020/11/26 20:34:27
episode : 35499 | final step : 98 | total reward : -1.0
2020/11/26 20:34:27
episode : 35500 | final step : 98 | total reward : -1.0
2020/11/26 20:34:28
episode : 35501 | final step : 98 | total reward : -1.0
2020/11/26 20:34:28
episode : 35502 | final step : 98 | total reward : -1.0
2020/11/26 20:34:29
episode : 35503 | final step : 98 | total reward : -1.0
2020/11/26 20:34:29
episode : 35504 | final step : 98 | total reward : -1.0
2020/11/26 20:34:30
episode : 35505 | final step : 98 | total reward : -1.0
2020/11/26 20:34:31
episode : 35506 | final step : 98 | total reward : -1.0
2020/11/26 20:34:31
episode : 35507 | final step : 98 | total reward : -1.0
2020/11/26 20:34:32
episode : 35508 | final step : 98 | total reward : -1.0
2020/11/26 20:34:32
episode : 35509 | final step : 98 | total reward : -1.0


In [ ]:
plt.figure(figsize=(15,10))
plt.xlabel("Episode")
plt.ylabel("Total rewards")
plt.plot(reward_list)

In [ ]:
# plots
Deg2Rad = np.pi/180
Rad2Deg = 1/Deg2Rad

plt_res=total_res[9999]


plt.figure(figsize=(15,9), dpi=100)

plt.subplot(511)
plt.plot(plt_res[:,0], label=r'$\dot{h}_{cmd}$')
plt.ylabel(r'$\dot{h}_{cmd}$ ($m/s$)'), plt.grid()

plt.subplot(512)
plt.plot(plt_res[:,10],label=r'$\{h}$')
plt.ylabel(r'$h$ (m)'), plt.grid()

plt.subplot(513)
plt.plot(plt_res[:,1],label=r'$\{r}$')
plt.ylabel(r'$r$ (m)'), plt.grid()

plt.subplot(514)
plt.plot(plt_res[:,2]*Rad2Deg, label='elevation')
plt.ylabel('elevation (deg)'), plt.grid()

plt.subplot(515)
plt.plot(plt_res[:,3]*Rad2Deg, label='azimuth')
plt.ylabel('azimuth (deg)'), plt.grid()

plt.legend()
plt.show()

In [ ]:
# trajectory plots

from mpl_toolkits.mplot3d import Axes3D

plt.figure(figsize=(12,9), dpi=100)
plt.gca(projection='3d')
plt.plot(plt_res[:,5], plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.xlim(-2000,2000)
plt.ylim(0,4000)
plt.legend()
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,5], plt_res[:,4], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('North')
plt.ylabel('Up')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()